In [1]:
subject = 'sample' 
atlas = 'dkt'

In [2]:
import os, sys
os.chdir('..')
from src.mmvt_addon.scripts import run_mmvt
run_mmvt.run(subject, atlas, run_in_background=False, debug=False, raise_exp=False)

usage: ipykernel_launcher.py [-h] [-s SUBJECT] [--subjects SUBJECTS]
                             [-a ATLAS] [--real_atlas REAL_ATLAS] [-b BIPOLAR]
                             [-d DEBUG]

MMVT

optional arguments:
  -h, --help            show this help message and exit
  -s SUBJECT, --subject SUBJECT
                        subject name
  --subjects SUBJECTS   subjects names
  -a ATLAS, --atlas ATLAS
                        atlas name
  --real_atlas REAL_ATLAS
                        atlas name
  -b BIPOLAR, --bipolar BIPOLAR
                        bipolar
  -d DEBUG, --debug DEBUG
                        debug
*********** sample ***********


error: unrecognized arguments: -f /run/user/3628004/jupyter/kernel-8698a0e0-4eb5-4497-99c7-d889b64847b3.json


In [4]:
from pizco import Proxy
mmvt = Proxy('tcp://127.0.0.1:8001');

In [5]:
import os.path as op
import mne

mne_sample_data_fol = mne.datasets.sample.data_path()
meg_subject = 'sample'
stc_fname = op.join(mne_sample_data_fol, 'MEG', 'sample', 'sample_audvis-meg-rh.stc')
stc = mne.read_source_estimate(stc_fname)

In [6]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Label, Button, Layout, Image

T = stc.lh_data.shape[1] 
play = widgets.Play(value=0, min=0, max=T, step=1, interval=1000)
time_slider = widgets.IntSlider(value=0, min=0, max=T, step=1)
widgets.jslink((play, 'value'), (time_slider, 'value'))
quality_slider = widgets.IntSlider(value=40, min=0, max=100, step=1, layout=Layout(width='100%'))
zoom_slider = widgets.IntSlider(value=0, min=-5, max=5, step=1, layout=Layout(width='100%'))
threshold_slider = widgets.FloatSlider(value=2, min=0, max=10, step=0.1, layout=Layout(width='20%'))

In [7]:
import numpy as np
def create_stc_t(stc, t):
    data = np.concatenate([stc.lh_data[:, t:t + 1], stc.rh_data[:, t:t + 1]])
    vertices = [stc.lh_vertno, stc.rh_vertno]
    stc_t = mne.SourceEstimate(data, vertices, stc.tmin + t * stc.tstep, stc.tstep, subject=subject)
    return stc_t

def plot_stc(stc, t):
#     return mmvt.plot_stc(stc, t, threshold=threshold_slider.value,  save_image=True)
    stc_t = create_stc_t(stc, t)
    stc_t_smooth = mne.morph_data(meg_subject, subject, stc_t, n_jobs=n_jobs, grade=vertices_to)
    meg_fname = mmvt.plot_stc_t(stc_t_smooth.rh_data, stc_t_smooth.lh_data, t, threshold=threshold_slider.value,
                                save_image=True, view_selected=False)
    mmvt.set_current_t(t)
    return meg_fname, stc_t_smooth
    
n_jobs = 4
t = 0 # ms
vertices_to = mne.grade_to_vertices(subject, None)    
meg_fname, stc_t_smooth = plot_stc(stc, t);
file = open(meg_fname, "rb")
meg_image = Image(value=file.read(), format='png', width=800, height=450)

Morphing data...
    Left-hemisphere map read.
    Right-hemisphere map read.
    Left-hemisphere map read.
    Right-hemisphere map read.
    17 smooth iterations done.


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.5s finished


    14 smooth iterations done.
[done]


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.4s finished


In [8]:
def update_image(fname=''):
    fname = mmvt.save_image('stc', index=time_slider.value) if fname == '' else fname
    file = open(fname, "rb")
    meg_image.value = file.read()


sagital_button = Button(description="Sagital")
sagital_button.on_click(lambda x:(mmvt.show_sagital(), mmvt.zoom(-1), update_image()))
coronal_button = Button(description="Coronal")
coronal_button.on_click(lambda x:(mmvt.show_coronal(), mmvt.zoom(-1), update_image()))
axial_button = Button(description="Axial")
axial_button.on_click(lambda x:(mmvt.show_axial(), mmvt.zoom(-1), update_image()))
zoom_button = Button(description="zoom")
zoom_button.on_click(lambda x:(mmvt.zoom(zoom_slider.value), update_image()))
render_button = Button(description='Render')
update_button = Button(description='Update')
update_button.on_click(lambda x:(update_image()))
render_button.on_click(lambda x:(mmvt.hide_subcorticals(), mmvt.set_brain_transparency(1), mmvt.set_to_camera_view(),
                                 update_image(mmvt.render_image(quality=quality_slider.value))))

In [9]:
def on_value_change(change):
    t = change['new']
    meg_fname, stc_t_smooth = plot_stc(stc, t)
    file = open(meg_fname, "rb")
    meg_image.value = file.read()

time_slider.observe(on_value_change, names='value') 
VBox([HBox([meg_image, VBox([sagital_button, coronal_button, axial_button, update_button,
                             #Label('Zoom:'), zoom_slider, zoom_button,
                             Label('Quality:'), quality_slider, render_button])]),
     HBox([play, time_slider, Label('Threshold:'), threshold_slider])])

Morphing data...
    Left-hemisphere map read.
    Right-hemisphere map read.
    Left-hemisphere map read.
    Right-hemisphere map read.
    17 smooth iterations done.


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.4s finished


    14 smooth iterations done.
[done]


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.4s finished


Morphing data...
    Left-hemisphere map read.
    Right-hemisphere map read.
    Left-hemisphere map read.
    Right-hemisphere map read.
    17 smooth iterations done.


...

[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.4s finished


Morphing data...
    Left-hemisphere map read.
    Right-hemisphere map read.
    Left-hemisphere map read.
    Right-hemisphere map read.
    17 smooth iterations done.


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.4s finished


    14 smooth iterations done.
[done]


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.4s finished
